# NFL 1st and Future - Analytics
### Can you investigate the relationship between the playing surface and the injury and performance of NFL athletes?
In this challenge, you're tasked to investigate the relationship between the playing surface and the injury and performance of National Football League (NFL) athletes and to examine factors that may contribute to lower extremity injuries.

You'll also notice there isn't a leaderboard, and you are not required to develop a predictive model. This isn't a traditional supervised Kaggle machine learning competition. For more information on this challenge format, see this forum thread. This challenge is part of NFL 1st & Future, the NFL’s annual Super Bowl competition designed to spur innovation in player health, safety and performance.

## About The NFL

The National Football League is America's most popular sports league, comprised of 32 franchises that compete each year to win the Super Bowl, the world's biggest annual sporting event. Founded in 1920, the NFL developed the model for the successful modern sports league, including national and international distribution, extensive revenue sharing, competitive excellence, and strong franchises across the country.

The NFL is committed to advancing progress in the diagnosis, prevention and treatment of sports-related injuries. The NFL's ongoing health and safety efforts include support for independent medical research and engineering advancements and a commitment to work to better protect players and make the game safer, including enhancements to medical protocols and improvements to how our game is taught and played.

As more is learned, the league evaluates and changes rules to evolve the game and try to improve protections for players. Since 2002 alone, the NFL has made 50 rules changes intended to eliminate potentially dangerous tactics and reduce the risk of injuries.

For more information about the NFL's health and safety efforts, please visit www.PlaySmartPlaySafe.com

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline

In [ ]:
games=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayList.csv')
injrec=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/InjuryRecord.csv')

# Data preparation
We need set up a data for good analyze and visualisation. At first, I removed the nan values. Others was replaced by nan. In the column PlayType Not Returnded and Returnded were removed to group the Punt and Kickoff PlayType.

In [ ]:
games.fillna({'StadiumType': 'NoData',
                     'Weather': 'NoData',
                     }, inplace=True)

games.fillna(method='ffill', inplace=True)

games.replace('Missing Data', np.nan, inplace=True)

games['Temperature'].replace(-999, np.nan, inplace=True)

games.PlayType=games.PlayType.str.replace('Not Returned', '').str.replace('Returned', '').str.strip()
games.PlayType.replace('0', np.nan, inplace=True)

There is a serious data problem in the Weather Column. It may be noticed that the same or similar values in this column have written differently, like Coudy, cloudy, Indoor, Indoors. Therefore, I decided to group all the weather values into several types: Sunny, Clear, Cloudy, Partly Cloudy, Controled Climate, Rain and Bad.

In [ ]:
games.Weather=games.Weather.replace({'Indoors': 'Controlled Climate', 'Rain': 'Rain and Bad', 
                                     'N/A (Indoors)': 'Controlled Climate', 'Snow': 'Rain and Bad',
                                     'Indoor': 'Controlled Climate', 'Overcast': 'Rain and Bad',
                                     'N/A Indoor': 'Controlled Climate', 'Clear and cold': 'Clear',
                                     'Mostly Cloudy': 'Cloudy', 'Sunny and clear': 'Sunny',
                                     'Mostly cloudy': 'Cloudy', 'Rain Chance 40%': 'Rain and Bad',
                                     'Cloudy and cold': 'Cloudy', 'Sunny, highs to upper 80s': 'Sunny',
                                     'Cloudy and Cool': 'Cloudy', 'Cloudy, light snow accumulating 1-3"': 'Rain and Bad',
                                     'Partly cloudy': 'Partly Cloudy', 'Scattered Showers': 'Rain and Bad',
                                     'Party Cloudy': 'Partly Cloudy', 'Cold': 'Rain and Bad',
                                     'Partly Clouidy': 'Partly Cloudy', 'Sunny and cold': 'Sunny', 
                                     'Mostly Coudy': 'Partly Cloudy', 'Partly sunny': 'Sunny',
                                     'cloudy': 'Cloudy', 'Cloudy, fog started developing in 2nd quarter ': 'Rain and Bad',
                                     'Coudy': 'Cloudy', 'Showers': 'Rain and Bad',
                                     'Mostly Sunny': 'Sunny', 'Rainy': 'Rain and Bad',
                                     'Partly Sunny': 'Sunny', 'Clear to Partly Cloudy': 'Partly Cloudy',
                                     'Mostly sunny': 'Sunny', 'Rain shower': 'Rain and Bad',
                                     'Sunny Skies': 'Sunny', 'Heat Index 95': 'Rain and Bad',
                                     'Sunny and warm': 'Sunny', 'Cloudy, Rain': 'Rain and Bad',
                                     'Clear and Sunny': 'Clear', 'Heavy lake effect snow': 'Rain and Bad',
                                     'Clear and sunny': 'Clear', '30% Chance of Rain': 'Rain and Bad', 
                                     'Mostly Sunny Skies': 'Sunny', 'Cloudy, chance of rain ': 'Rain and Bad',
                                     'Clear Skies': 'Clear', 'Cloudy, 50% change of rain': 'Rain and Bad',
                                     'Clear skies': 'Clear', 'Rain likely, temps in low 40s.': 'Rain and Bad',
                                     'Clear and Cool': 'Clear', 'Sunny, Windy': 'Sunny',
                                     'Fair': 'Clear', 'Cloudy, fog started developing in 2nd quarter': 'Rain and Bad',
                                     'Light Rain': 'Rain and Bad', '10% Chance of Rain': 'Rain and Bad',
                                     'Clear and warm': 'Clear', 'Cloudy, chance of rain': 'Cloudy',
                                     'Hazy': 'Cloudy', 'Partly clear': 'Clear', 'Sun & clouds': 'Sunny',
'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.': 'Rain and Bad'})     

In [ ]:
games.Weather=games.Weather.replace({'Controlled Climate': 'CC',
                                    'Rain and Bad': 'RaB',
                                    'Clear': 'Cr',
                                    'Sunny': 'Sun',
                                    'Partly Cloudy': 'PC',
                                    'Cloudy': 'Cl',
                                    'NoData': 'ND'})

The next step was to combine the data from the two tdata frames. Since, there was not full data in the data frame with injuries, merge gave out only 77 rows, which was wrong and needed to be fixed.

In [ ]:
injrec_full=pd.merge(games, injrec)
injrec_full.shape

First of all, I created data frame with all injury data using the "right" merge. Then, I grouped all injuries in one column and saw missed data.

In [ ]:
injrec_full=pd.merge(games, injrec, how='right')
injrec_full['Injuries']=injrec_full[['DM_M1', 'DM_M7', 'DM_M28', 'DM_M42']].sum(axis=1)
inj_dict={1: 'Fine', 2: 'Little problem', 3: 'Serious Trouble', 4: 'Catastrophic Event'}
injrec_full['Injuries']=injrec_full['Injuries'].map(inj_dict)
injrec_full.drop(['DM_M1','DM_M7', 'DM_M28', 'DM_M42'], axis=1, inplace=True)
injrec_full.tail(3)

Because there were no values in PlayKey, data frames were not be able to merge. Therefore, to get the full data I needed to find another way. We know the GameID and can assume that the injury occurred in the last PlayKey. Let's make a loop that will form a data frame with the necessary data.

In [ ]:
miss_data=games.loc[games['GameID']=='33337-2'].tail(1)
for i in range(78, len(injrec_full)):
    miss_data=miss_data.append(games.loc[games['GameID']==injrec_full.iloc[i][1]].tail(1))

But the merge didn't work with this data frame, so again I needed another way. I used append and then used function.

In [ ]:
miss_data_new=pd.merge(miss_data, injrec_full)
miss_data_new.shape

In [ ]:
injrec_full=injrec_full.append(miss_data, sort=False)

def get_column_array(injrec_full, column):
    expected_length = len(injrec_full)
    current_array = injrec_full[column].dropna().values
    if len(current_array) < expected_length:
        current_array = np.append(current_array, [''] * (expected_length - len(current_array)))
    return current_array

injrec_full=pd.DataFrame({column: get_column_array(injrec_full, column) for column in injrec_full.columns})

injrec_full.replace('', np.nan, inplace=True)
injrec_full=injrec_full.dropna(thresh=10)
injrec_full.tail(3)

I got the desired data frame, but still needed to fix the type of columns that had changed from manipulation.

In [ ]:
injrec_full['Temperature']=injrec_full['Temperature'].astype(float)
injrec_full['PlayerDay']=injrec_full['PlayerDay'].astype(float)
injrec_full['PlayerGame']=injrec_full['PlayerGame'].astype(float)
injrec_full['PlayerGamePlay']=injrec_full['PlayerGamePlay'].astype(float)

# Visualization
## Injuries
Finally, I began to visualize the data. According to the graphs, the ratio of injuries and the number of games by field type showed that the synthetic surface is 1.7 times more dangerous than natural.

In [ ]:
sns.set(rc={'axes.labelsize':30,
            'figure.figsize':(13.7,10.27),
            'xtick.labelsize':25,
            'ytick.labelsize':20})

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].pie(injrec['Surface'].value_counts(),
           autopct='%1.0f%%',
          colors=['#CDC70A', '#2A741B'],
          textprops={'fontsize': 18},
          shadow=True)

axs[1].pie(games['FieldType'].value_counts(),
           autopct='%1.0f%%',
           startangle=180,
          colors=['#2A741B', '#CDC70A'],
           textprops={'fontsize': 18},
          shadow=True)

axs[0].set_title('Distribution of Injuries by Surface',
                    fontsize= 24)

axs[1].set_title('Distribution of games by Surface',
                     fontsize= 24)

axs[1].legend(labels=games['FieldType'].value_counts().index,
    loc='upper right',
    prop={'size': 18},
     bbox_to_anchor=(1.4, 0.9),
             title='Surface',
             title_fontsize='28')

plt.show()

On the synthetic surface, there are more Serious and Catastrophic injuries.


In [ ]:
chart=sns.countplot(x='Surface',hue='Injuries',data=injrec_full, 
              palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],
             edgecolor=(0,0,0),
                  linewidth=2)

chart.legend(prop={'size': 22},
     bbox_to_anchor=(1, 0.9),
            title='Injuries',
            title_fontsize='35')

chart.set_title('Surface Injury Distribution',
               fontsize=40)

chart.set(yticks=range(0, 21,2))

plt.show()

The synthetic surface has more ankle and toes injuries.

In [ ]:
chart=sns.catplot(x='BodyPart', hue='Injuries', data=injrec_full, col='Surface', kind='count',
                  palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)

chart.set_xticklabels(size=20)

chart.set_xlabels(size=24)

chart.set(yticks=range(0, 13,2))

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='30')

plt.show()

## Visual Injuries by Play Type
It can be seen that Rush and Puss play types have the most injuries. But for the final conclusions we need to look at the game ratio.

In [ ]:
chart1=sns.countplot(x='PlayType', hue='Injuries', data=injrec_full, 
              palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],
             edgecolor=(0,0,0),
                  linewidth=2)

chart1.set_title('Distribution of Injuries by Play Type',
                     fontsize= 38)

chart1.legend(prop={'size': 22},
     bbox_to_anchor=(1, 0.9), title='Injuries',
            title_fontsize='35')

plt.show()

Based on the number of games, it is clear that Kickoff and Punt play types are more dangerous.

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].pie(injrec_full['PlayType'].value_counts(),
           radius=1.25,
           autopct='%1.0f%%',
           explode=(0, 0, 0.1, 0.2, 0),
           colors=['#094275', '#FA7D0F', '#0BA006', '#FF4513', '#DDE30F'],
           textprops={'fontsize': 18},
           shadow=True)

axs[1].pie(games['PlayType'].value_counts(),
           radius=1.25,
           autopct='%1.0f%%',
           explode=(0, 0, 0.2, 0.3, 0, 0),
           #startangle=180,
           colors=['#094275', '#FA7D0F', '#FF4513','#0BA006', '#7536A7', '#DDE30F'],
           textprops={'fontsize': 18},
           shadow=True)

axs[0].set_title('Distribution of Injuries by Play Type',
                     fontsize= 22)

axs[1].set_title('Distribution of games by Play Type',
                     fontsize= 22)

axs[1].legend(labels=games['PlayType'].value_counts().index,
    loc='upper right',
    prop={'size': 18},
     bbox_to_anchor=(1.6, 0.9),
             title='Play Type',
             title_fontsize='28')

plt.show()

All play types take more injuries on the synthetic surface.

In [ ]:
chart=sns.catplot(x='PlayType', hue='Injuries', data=injrec_full, col='Surface', kind='count',
                  palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)

chart.set_xticklabels(size=15)

chart.set_xlabels(size=24)

chart.set(yticks=range(0, 13,2))

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='30')

plt.show()

On the synthetic surface Pass play type get more ankle injuries.

In [ ]:
chart=sns.catplot(x='PlayType', hue='BodyPart', data=injrec_full, col='Surface', kind='count',
                  palette=['#272A7C', '#1696C6', '#1E7928', '#CCE914', '#FF0000'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)

chart.set_xticklabels(size=15)

chart.set_xlabels(size=24)

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='30')

chart.set(yticks=range(0, 15,2))

plt.show()

## Position Group
By the distribution of injuries and the total number of games, the safe group position OL is clearly visible. The most dangerous positions are WR, RB, TE

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].pie(injrec_full['PositionGroup'].value_counts(),
           radius=1.2,
           autopct='%1.0f%%',
           explode=(0, 0.1, 0, 0.2, 0, 0.2, 0.1),
          colors=['#8E08FE' ,'#258187', '#A525B9', '#D6D676', '#D11C4D', '#2B8138', '#C49144'],
           textprops={'fontsize': 18},
          shadow=True)

axs[1].pie(games['PositionGroup'].value_counts(),
           radius=1.2,
           autopct='%1.0f%%',
            explode=(0, 0, 0.1, 0.1, 0, 0.25, 0.1, 0, 0),
          colors=['#8E08FE', '#A525B9', '#2B8138','#258187', '#D11C4D', '#D6D676', '#C49144', '#F96905', '#D1BFB2'],
           textprops={'fontsize': 18},
          shadow=True)

axs[0].set_title('Position Group Injury Distribution',
                     fontsize= 24)

axs[1].set_title('Position Group Games Distribution',
                     fontsize= 24)

axs[1].legend(labels=games['PositionGroup'].value_counts().index,
    loc='upper right',
    prop={'size': 16},
     bbox_to_anchor=(1.7, 0.9),
             title='Position Group',
             title_fontsize='28')

plt.show()

In [ ]:
chart1=sns.countplot(x='PositionGroup', hue='Injuries', data=injrec_full, 
              palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],
             edgecolor=(0,0,0),
                  linewidth=2)

chart1.set_title('Distribution of Injuries by Position Group',
                     fontsize= 38)

chart1.legend(prop={'size': 22},
     bbox_to_anchor=(1, 0.9), title='Injuries',
            title_fontsize='35')

plt.show()

DB and WR position groups are more dangerous on the synthetic field, but for OL position synthetic surface is really good.

In [ ]:
chart=sns.catplot(x='PositionGroup', hue='Injuries', data=injrec_full, col='Surface', kind='count',
                  palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)

chart.set_xticklabels(size=15)

chart.set_xlabels(size=24)

chart.set(yticks=range(0, 13,2))

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='30')

plt.show()

DB has more ankle injuries on the synthetic field.

In [ ]:
chart=sns.catplot(x='PositionGroup', hue='BodyPart', data=injrec_full, col='Surface', kind='count',
                  palette=['#272A7C', '#1696C6', '#1E7928', '#CCE914', '#FF0000'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)
                  #height=15, aspect=1)
    
chart.set_xticklabels(size=15)

chart.set_xlabels(size=24)

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='30')

chart.set(yticks=range(0, 13,2))

plt.show()

## Weather
Sunny, partly cloudy and clear weather are more dangerous. The safest weather is cloudy.

In [ ]:
wth=['Cloudy', 'Sunny', 'Clear', 'Partly Cloudy','Rain and Bad', 'Controlled Climate', 'NoData']

fig, axs = plt.subplots(1, 2)

axs[0].pie(injrec_full['Weather'].value_counts(),
           radius=1.2,
           autopct='%1.0f%%',
         explode=(0.1, 0, 0.1, 0, 0.2, 0.2, 0),
          colors=['#F7FE00', '#498484', '#1550A0', '#B8CCF2', '#DE00FF', '#D2355D', '#453547' ],
           textprops={'fontsize': 18},
          shadow=True)

axs[1].pie(games['Weather'].value_counts(),
           radius=1.2,
           autopct='%1.0f%%',
            explode=(0.1, 0, 0.1, 0, 0.2, 0.2, 0),
         colors=['#498484', '#F7FE00', '#B8CCF2','#1550A0', '#D2355D', '#DE00FF', '#453547'],
           textprops={'fontsize': 18},
          shadow=True)

axs[0].set_title('Weather Injury Distribution',
                     fontsize=24)
axs[1].set_title('Weather Games Distribution',
                     fontsize=24)

axs[1].legend(labels=wth,
    loc='upper right',
    prop={'size': 20},
     bbox_to_anchor=(1.8, 0.9),
             title='Weather',
             title_fontsize='28')

plt.show()

In [ ]:
chart1=sns.countplot(x='Weather', hue='Injuries', data=injrec_full, 
              palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],
             edgecolor=(0,0,0),
                  linewidth=2)

chart1.set_title('Distribution of Injuries by Position Group',
                     fontsize= 38)

chart1.legend(prop={'size': 25},
     bbox_to_anchor=(1, 0.9), title='Injuries',
            title_fontsize='45')

plt.show()

In [ ]:
chart=sns.catplot(x='Weather', hue='Injuries', data=injrec_full, col='Surface', kind='count', 
                  palette=['#820108', '#FF4C56', '#EEF10B', '#0BF1CE'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)

chart.set_xticklabels(size=20)

chart.set_xlabels(size=24)

chart.set(yticks=range(0, 13,2))

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='80')

plt.show()

In [ ]:
chart=sns.catplot(x='Weather', hue='BodyPart', data=injrec_full, col='Surface', kind='count',
                  palette=['#272A7C', '#1696C6', '#1E7928', '#CCE914', '#FF0000'],edgecolor=(0,0,0),
                  linewidth=2, legend=False)

chart.set_xticklabels(size=20)

chart.set_xlabels(size=24)

chart.set(yticks=range(0, 13,2))

chart.add_legend(prop={'size': 18},
            title='Injuries',
            title_fontsize='30')

plt.show()

## Comparisons

In [ ]:
labels=['Temperature', 'PlayerDay', 'PlayerGame', 'PlayerGamePlay']
x=injrec_full[['Temperature', 'PlayerDay', 'PlayerGame', 'PlayerGamePlay']].mean().astype(int)
y=games[['Temperature', 'PlayerDay', 'PlayerGame', 'PlayerGamePlay']].mean().astype(int)
z = np.arange(len(x))  
width = 0.35  

fig, ax = plt.subplots()

rects1 = ax.bar(z - width/2, x, width, label='Injuries games', edgecolor=(0,0,0),
                  linewidth=2)
rects2 = ax.bar(z + width/2, y, width, label='All Games', edgecolor=(0,0,0),
                  linewidth=2)

ax.set_ylabel('Count')
ax.set_title('Average Comparison', fontsize=26)
ax.set_xticks(z)
ax.set_xticklabels(labels)
ax.legend( loc='upper right',
    prop={'size': 20})

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 4), 
                    textcoords="offset points",
                    ha='center', va='bottom', size=20)
autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
fig.set_size_inches(12, 8)

plt.show()

In [ ]:
labels=['Temperature', 'PlayerDay', 'PlayerGame', 'PlayerGamePlay']
x=injrec_full[['Temperature', 'PlayerDay', 'PlayerGame', 'PlayerGamePlay']].median().astype(int)
y=games[['Temperature', 'PlayerDay', 'PlayerGame', 'PlayerGamePlay']].median().astype(int)
z = np.arange(len(x))  
width = 0.35 

fig, ax = plt.subplots()

rects1 = ax.bar(z - width/2, x, width, label='Injuries games', edgecolor=(0,0,0),
                  linewidth=2)
rects2 = ax.bar(z + width/2, y, width, label='All games', edgecolor=(0,0,0),
                  linewidth=2)

ax.set_ylabel('Count')
ax.set_title('Median Comparison',  fontsize=26)
ax.set_xticks(z)
ax.set_xticklabels(labels)
ax.legend(loc='upper right',
    prop={'size': 20})

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), 
                    textcoords="offset points",
                    ha='center', va='bottom',
                    size=20)
autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
fig.set_size_inches(12, 8)

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.suptitle('Distribution of Injured games and all games by', fontsize=36)

ax1=plt.subplot(221)
sns.kdeplot(injrec_full['PlayerDay'], shade=True, color='#FF0000', legend=False)
sns.kdeplot(games['PlayerDay'], shade=True, color='#002CF2', legend=False)
ax1.set_title('Player Day', fontsize=24)

ax2=plt.subplot(2, 2, 2)
sns.kdeplot(injrec_full['PlayerGame'], shade=True, color='#FF0000', label='With Injuries')
sns.kdeplot(games['PlayerGame'], shade=True, color='#002CF2', label='All')
plt.legend(loc='upper right',
           bbox_to_anchor=(1.75, 0.9),
    prop={'size': 24},
          title='Games',
             title_fontsize='28')
ax2.set_title('Player Game', fontsize=24)

ax3=plt.subplot(2, 2, 3)
sns.kdeplot(injrec_full['PlayerGamePlay'], shade=True, color='#FF0000', legend=False)
sns.kdeplot(games['PlayerGamePlay'], shade=True, color='#002CF2', legend=False)
ax3.set_title('Player Game Play', fontsize=24)

ax4=plt.subplot(2, 2, 4)
sns.kdeplot(injrec_full['Temperature'], shade=True, color='#FF0000', legend=False)
sns.kdeplot(games['Temperature'], shade=True, color='#002CF2', legend=False)
ax4.set_title('Temperature', fontsize=24)

plt.show()

# Conclusion
1.1. Synthetic surface is 1.7 times more dangerous than natural one.

1.2. The knee and ankle are most prone to injury. At the same time, the risk of ankle injury is significantly higher on the synthetic surface (8 vs 3). Knee injuries are almost the same on the both surfaces. Also, synthetic coating is much more dangerous for the toe. (6 vs 1)

1.3. But it is worth noting that catastrophic (M42) foot injuries are clearly higher on natural surfaces (1 vs. 4).

2.1. Most injured players Rush and Pass play type, but this is due to their advantage in the number of games. Based on the game to injury ratio, Punt and Kickoff players have more injuries. With 4% and 5% of the total number of games, the number of injuries is 14% and 11%, respectively.

2.2. The synthetic surface is more dangerous for kickoff, since it gets more injuries on it (8 vs 4).

2.3. Punt receives more injuries on the natural field (9 vs. 6), but they are mostly light, and the synthetic  injuries are dominated by severe injuries 3 catastrophic(M42) and 1 serious injury(M28).

2.4. Rush and Pass players have greater risk of injury on the synthetic field, especially serious (M28) injuries (5 vs 2).

2.5. The main injury of kickoff and punt players is on their knees (18 of 27).

2.6.  The ankles and knees are almost subject  of the same danger (14 and 15 injuries) for rush and pass players on the natural field. The ankles are more at risk (21 injuries), even the knees are slightly smaller on the synthetic field than on the natural field (13 injuries).

3.1. The most traumatic position - group Db, because it has the most games. Based on the game/injury ratio, the most dangerous position is RB. With 4% of games, it accounts for 9% of injuries. WR (16% of games and 22% of injuries), LB (19% of games and 22% of injuries) and TE (3% of games and 6% of injuries) also dangerous.

3.2. The safest position is OL. With 18% of games, only 7% of injuries occure. QB is also not very dangerous . There are no injuries at 3% of games.

3.3. The distribution of injuries is pretty similar across all roles. Mostly catastrophic injuries (M42) are in the RB position, and a large number of minor injuries (M7) are in the DB stand out.

3.4. By type of field, the DB gets more injuries on the synthetic field (18 vs 10), and WR is more catastrophic (4 vs 1).

3.5. An interesting point is that the OL position on the synthetic field got only light injuries (M1), and on the natural surface, there is as much as 2 catastrophic (M42). Otherwise, the positions are not much different.

3.6. By injury to a part of the body, only the DB role has a strong difference. Many ankle injuries are on the synthetic field (11 vs 2).

4.1. An interesting weather observation that the most dangerous is sunny, clear and partially cloudy weather, which also confirms the comparison of average and median temperatures. Moreover, there are the greatest number of catastrophic injuries in sunny weather (M42) (10).

4.2. Bad weather and climate-controlled are also a little dangerous. With 8% of all games, injuries are 11%.

4.3. And much safer is cloudy weather. With 29% of all injury games, it accounts for only 17%, which is almost two times less.

4.4. In sunny weather, injuries end more disastrously (M42) on the natural field than on the artificial one. (6 vs 4).  The risk of minor (M7) injury on the natural field is much greater than on the synthetic one (12 vs 4) in clean and bad weather. 

4.5. There are mostly not serious injuries (6 light(M1), 4 minor (M7) and only 2 catastrophic (M42)) in the controlled climate, which is only on synthetic surface.  From this, I can conclude that in sunny weather with high temperature it is more reasonable to conduct a match on the synthetic surface with a controlled climate.

5.1. The comparison of the average and median values of the number of games and days shows that the amount does not affect the risk of injury. On the contrary, all injuries are more likely in the middle of the season.

5.2. The comparison of player game play shows that their number does not affect the risk of injury. On the contrary, if a player plays the danger zone (15-25), then the farther he plays the less chance of injury.